# Data check
We will take a look at data sent in January 2024

## The notebooks in this folder deal with new datasets. This is a temporary notebok to discuss the data we have

## There are a few irregularities to discuss

### Load libraries

In [ ]:
import os       # using operating system dependent functionality (folders)
import sys
import glob
from functools import reduce

import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive
import seaborn as sns

sys.path.insert(0, '../') # path to functions
import cvasl.seperated as sep
from cvasl.file_handler import Config
import cvasl.harmony as har


In [ ]:
# Datasets for this work
EDIS_path = 'our_datasets/EDIS/'
HELIUS_path = 'our_datasets/HELIUS/'
Insight46_path = 'our_datasets/Insight46/'
SABRE_path = 'our_datasets/SABRE/'
MRI_path = 'our_datasets/StrokeMRI/'
TOP_path = 'our_datasets/TOP/'
file_name = 'TrainingDataComplete.csv'

TOP_file = os.path.join(TOP_path, file_name)
MRI_file = os.path.join(MRI_path, file_name)
EDIS_file = os.path.join(EDIS_path, file_name)
#HELIUS_file = os.path.join(HELIUS_path, file_name)
Insight46_file = os.path.join(Insight46_path, file_name)
SABRE_file = os.path.join(SABRE_path, file_name)

EDIS = pd.read_csv(EDIS_file)
#HELIUS = pd.read_csv(HELIUS_file)
Insight46 = pd.read_csv(Insight46_file)
SABRE = pd.read_csv(SABRE_file)
TOP = pd.read_csv(TOP_file)
MRI = pd.read_csv(MRI_file)

In [ ]:
#example old datasets
filepath_mri_old = '../open_work/internal_results/cleaned_pvc2s/' 
filename_mri_old = os.path.join(filepath_mri_old,'StrokeMRI_pvc2c.csv') 
filepath_top_old = '../open_work/internal_results/cleaned_pvc2s/' 
filename_top_old = os.path.join(filepath_top_old,'TOP_pvc2c.csv') 
TOP_old = pd.read_csv(filename_top_old)
StrokeMRI_old = pd.read_csv(filename_mri_old)
TOP_old = TOP_old.drop(TOP_old.columns[0],axis=1)
StrokeMRI_old = StrokeMRI_old.drop(StrokeMRI_old.columns[0],axis=1)

# Now we need to flip the sex back to numbers for a correlation
# sex_mapping = {'F':0,'M':1}
# TOP = TOP.assign(sex = TOP.sex.map(sex_mapping))
# StrokeMRI = StrokeMRI.assign(sex = StrokeMRI.sex.map(sex_mapping))
#TOP_old.head(3)

In [ ]:
TOP.shape

In [ ]:
TOP.columns

In [ ]:
## SO one minor note...this is not what we agreed to. We agreed to everything lower case. We can fix this with a re funciton to lower case everything

In [ ]:
# make every column lower case
TOP.columns = TOP.columns.str.lower() 

In [ ]:
set_top_columns = set(TOP.columns)


In [ ]:
## But let's compare to old dataset

In [ ]:
set_old_top_columns = set(TOP_old.columns)

In [ ]:
# what was added new in this version of top
set_top_columns - set_old_top_columns 

In [ ]:
# what we lost in this version of top
set_old_top_columns - set_top_columns

In [ ]:
# we must discuss this with scientists